In [1]:
import mlflow
import mlflow.sklearn

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn. metrics import *
import matplotlib.pyplot as plt

In [3]:
df_credit = pd.read_csv('Credit.csv')
df_credit.shape

(1000, 21)

In [4]:
df_credit.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,'critical/other existing credit',radio/tv,1169,'no known savings',>=7,4,'male single',none,...,'real estate',67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,'existing paid',radio/tv,5951,<100,1<=X<4,2,'female div/dep/mar',none,...,'real estate',22,none,own,1,skilled,1,none,yes,bad
2,'no checking',12,'critical/other existing credit',education,2096,<100,4<=X<7,2,'male single',none,...,'real estate',49,none,own,1,'unskilled resident',2,none,yes,good
3,<0,42,'existing paid',furniture/equipment,7882,<100,4<=X<7,2,'male single',guarantor,...,'life insurance',45,none,'for free',1,skilled,2,none,yes,good
4,<0,24,'delayed previously','new car',4870,<100,1<=X<4,3,'male single',none,...,'no known property',53,none,'for free',2,skilled,2,none,yes,bad


In [5]:
#Transformar de categorico para numerico
for col in df_credit.columns:
    if df_credit[col].dtype == 'object':
        df_credit[col] = df_credit[col].astype('category').cat.codes
            

In [6]:
df_credit.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,2,6,1,7,1169,0,3,4,3,2,...,2,67,1,1,2,3,1,1,1,1
1,1,48,3,7,5951,3,0,2,0,2,...,2,22,1,1,1,3,1,0,1,0
2,0,12,1,4,2096,3,1,2,3,2,...,2,49,1,1,1,2,2,0,1,1
3,2,42,3,5,7882,3,1,2,3,1,...,0,45,1,0,1,3,2,0,1,1
4,2,24,2,1,4870,3,0,3,3,2,...,1,53,1,0,2,3,2,0,1,0


In [7]:
X = df_credit.iloc[:, 0:20].values
y = df_credit.iloc[:, 20].values

In [8]:
teste = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = teste, random_state=123)

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((700, 20), (300, 20), (700,), (300,))

### Definindo e registrando as metricas

In [ ]:
mlflow.set_experiment('nbexperimento')

with mlflow.start_run():
    n_b = GaussianNB()
    n_b.fit(X_train, y_train)
    previsoes = n_b.predict(X_test)

    #métricas
    acuracia = accuracy_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    precision = precision_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    auc = roc_auc_score(y_test, previsoes)
    log = log_loss(y_test, previsoes)
    
    #regitros de métricas
    mlflow.log_metric('acuracia', acuracia)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('precision', precision)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('log', log)
    
    #gerar gráficos
    confusion = plot_confusion_matrix(n_b, X_test, y_test) 
    plt.savefig('confusion.png')
    roc = plot_roc_curve(n_b, X_test, y_test)
    plt.savefig('roc.png')
    
    
    #acessar gráficos
    mlflow.log_artifact('confusion.png')
    mlflow.log_artifact('roc.png')
    
    #registro do modelo
    mlflow.sklearn.log_model(n_b, 'Modelonb')
    
    
    
    #informações da execução
    print('Modelo: ', mlflow.active_run().info.run_uuid)
mlflow.end_run()    